# Submission Validation
## for Project Aria 3D Object Detection Challenges

This tutorial will load the example submission file and visualize it. Participants can use this notebook to validate their submission files before submitting to the eval.ai website. 

### Notebook stuck?
Note that because of Jupyter and Plotly issues, sometimes the code may stuck at visualization. We recommend **restart the kernels** and try again to see if the issue is resolved.

In [ ]:
import os
import matplotlib.pyplot as plt
import zipfile
import numpy as np

from projectaria_tools.core.stream_id import StreamId
from projectaria_tools.projects.adt import (
   AriaDigitalTwinDataProvider,
   AriaDigitalTwinDataPathsProvider,
)
from projectaria_tools.projects.adt.utils import (
    VERTICES_AXIS, 
    get_timed_homo_poses,
)

### Select a phase, a submission file, and a sequence for visualization

In [ ]:
phase = "example"
example_root = os.path.expanduser('~') + "/Documents/bug_bash_adt"

sequence_name = "Apartment_release_challenge_example"
sequence_folder = os.path.join(example_root, sequence_name)
submission_file = os.path.join(sequence_folder, "submissions_annotations_example", "annotations_example.zip")

### Load the sequence

In [ ]:
paths_provider = AriaDigitalTwinDataPathsProvider(sequence_folder)
selected_device_number = 0
data_paths = paths_provider.get_datapaths_by_device_num(selected_device_number)

gt_provider = AriaDigitalTwinDataProvider(data_paths)

### Load all predicted poses and timestamps

In [ ]:
submission_archive = zipfile.ZipFile(
    submission_file, mode="r", compression=zipfile.ZIP_BZIP2
)
predicted_poses = get_timed_homo_poses(
    submission_archive, 
    sequence_name
)

rgb_streamid = "214-1"
stream_id = StreamId(rgb_streamid)
img_timestamps_ns = gt_provider.get_aria_device_capture_timestamps_ns(stream_id)

### Visualize the predicted pose on the image given a timestamp

In [ ]:

# choose the frame in the middle of the sequence
select_timestamps_ns = img_timestamps_ns[50]

image_with_dt = gt_provider.get_aria_image_by_timestamp_ns(select_timestamps_ns, stream_id)

# check image is valid. It's always possible that the data retrieval fails, therefore all
# returned data not only contains dt, but also contains an isValid() function, or returns
# an optional variable.
assert image_with_dt.is_valid(), "Image not valid!"

# convert image to numpy array
image = image_with_dt.data().to_numpy_array()

plt.figure(figsize=[6, 6])
plt.imshow(image)
plt.axis("off")

# get the Aria pose
aria3dpose_with_dt = gt_provider.get_aria_3d_pose_by_timestamp_ns(select_timestamps_ns)
if not aria3dpose_with_dt.is_valid():
    print("aria 3d pose is not available")
aria3dpose = aria3dpose_with_dt.data()

# now to project 3D bbox to Aria camera
# get 6DoF object pose with repect to the target camera
transform_cam_device = gt_provider.get_aria_transform_device_camera(stream_id).inverse()
transform_cam_scene = np.matmul(
    transform_cam_device.to_matrix(), aria3dpose.transform_scene_device.inverse().to_matrix()
)

# get projection function
cam_calibration = gt_provider.get_aria_camera_calibration(stream_id)
assert cam_calibration is not None, "no camera calibration"

for proto, poses in predicted_poses[select_timestamps_ns].items():
    for transform_scene_obj in poses:
        transform_cam_obj = transform_cam_scene @ transform_scene_obj
        # Draw Axis
        vertices_i = []
        skip = False
        for vertice_o in VERTICES_AXIS:
            vertice_c = transform_cam_obj[:3, :3] @ vertice_o + transform_cam_obj[:3, 3]
            vertice_i = cam_calibration.project(vertice_c)
            if vertice_i is None:
                skip = True
            else:
                vertices_i.append(vertice_i)
        if skip:
            print(f"{proto} is skipped for visualization due to outside of image.")
            continue
        vertices_i = np.stack(vertices_i)
        plt.plot(
            [int(vertices_i[0, 0]), int(vertices_i[1, 0])],
            [int(vertices_i[0, 1]), int(vertices_i[1, 1])],
            c="r",
        )
        plt.plot(
            [int(vertices_i[0, 0]), int(vertices_i[2, 0])],
            [int(vertices_i[0, 1]), int(vertices_i[2, 1])],
            c="g",
        )
        plt.plot(
            [int(vertices_i[0, 0]), int(vertices_i[3, 0])],
            [int(vertices_i[0, 1]), int(vertices_i[3, 1])],
            c="b",
        )
        plt.text(
            int(vertices_i[0, 0]), int(vertices_i[0, 1]), proto, c="y", fontsize=6
        )
